 【問題1】コンペティション内容の確認
コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。


・何を学習し、何を予測するのか

・どのようなファイルを作りKaggleに提出するか#

・提出されたものはどういった指標値で評価されるのか

In [ ]:
# 各種インポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
# ローカルからデータ読み込み
df = pd.read_csv("application_train.csv")
print(df.head())

   SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   ...  FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG_DOCUMENT_21  \
0  ...                 0             

### 何を学習し、何を予測するのか

顧客の返済能力を学習し、債務不履行リスクを予測する。

### どのようなファイルを作りKaggleに提出するか
### 提出されたものはどういった指標値で評価されるのか


提出物は 、予測された確率と観測されたターゲットの間の受信機動作特性　 ＲＯＣ曲線下面積で評価されます。

提出ファイル

SK_ID_CURR テストセットのそれぞれについて、TARGET変数の確率を予測する必要があります。ファイルにはヘッダーが含まれ、次の形式である必要があります。


```
SK_ID_CURR,TARGET
100001,0.1
100005,0.9
100013,0.2
etc.
```

まずは単純な方法による ベースラインモデル　を作成します。精度の基準となるモデルです。


精度を高くする必要はありません。エラーなく実行でき、Kaggle側に推定値を提出できることを目指します。


# 【問題2】学習と検証
データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。


検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。



In [13]:
# カテゴリ変数をラベルエンコーディング
# カテゴリ変数(dtype=objectの列)を抜き出す

def _get_categorical_features(df):
    feats = [col for col in list(df.columns) if df[col].dtype == 'object']
    return feats

obj = _get_categorical_features(df)
print(obj)

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']


In [14]:
# カテゴリ変数を数値化
def _factorize_categoricals(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df

_factorize_categoricals(df, obj)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,0,0,0,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,1,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,0,1,0,0,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,0,1,0,0,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,0,1,0,0,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# 欠損値の削除
#df.fillna(df.mean(), inplace=True)

KeyboardInterrupt: 

In [31]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

In [32]:
X

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,100002,0,0,0,0,0,202500.0,406597.5,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0,1,0,1,0,270000.0,1293502.5,1,1,...,0,0,0,0,0,0,0,0,0,0
2,100004,1,0,1,0,0,67500.0,135000.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,1,0,0,0,135000.0,312682.5,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,0,0,0,0,121500.0,513000.0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,1,0,157500.0,254700.0,0,0,...,0,0,0,0,0,0,0,0,0,0
307507,456252,0,1,0,0,0,72000.0,269550.0,0,3,...,0,0,0,0,0,0,0,0,0,0
307508,456253,0,1,0,0,0,153000.0,677664.0,0,0,...,0,0,0,0,0,0,0,0,0,0
307509,456254,0,1,0,0,0,171000.0,370107.0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# データ分割
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( X, y, train_size=0.75, random_state=50)
print(x_train, x_test, y_train, y_test)

        SK_ID_CURR  NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  \
218277      352885                   1            0             1   
241863      380052                   0            0             0   
211345      344914                   0            1             0   
287077      432515                   0            1             0   
169751      296734                   0            0             1   
...            ...                 ...          ...           ...   
165959      292395                   0            0             1   
186463      316164                   0            0             0   
153709      278148                   0            1             1   
239499      377348                   0            1             0   
103904      220588                   0            1             1   

        FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
218277                1             2          225000.0    675000.0   
241863                1      

In [34]:
# 最近傍法で学習

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)

In [41]:
print(y_test[100:150]==y_pred[100:150])

170641     True
125536    False
248569     True
242842     True
153450    False
257719     True
148812     True
54796      True
223894     True
226978     True
56035      True
263255     True
293037     True
173546     True
88974      True
10224      True
2696      False
162930     True
24409      True
241692     True
5253       True
38193      True
277046     True
33470      True
167932     True
35348      True
175457     True
289650     True
184021     True
284817     True
197080     True
68182      True
245853     True
305312     True
183574     True
38696      True
305688     True
100307     True
165911     True
305772     True
242539     True
97248      True
108774     True
239112     True
30         True
149903     True
26555      True
12211      True
130025     True
31064      True
Name: TARGET, dtype: bool


In [69]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

outlist = {}

def output(y, model, label,outlist={}):
    cm = confusion_matrix(y, model)
    ac = metrics.accuracy_score(y, model)
    pr = metrics.precision_score(y, model)
    rc = metrics.recall_score(y, model)
    f1 = metrics.f1_score(y, model)

    print('confusion matrix = \n', cm)
    print('accuracy = ', ac)
    print('precision = ', pr)
    print('recall = ', rc)
    print('f1 score = ', f1)

    outlist[label] = [cm, ac, pr, rc, f1]
    return outlist

d = output(y_test, y_pred, "KNN")
tmp = pd.DataFrame(d.values(), index=d.keys(), columns= ["confusion matrix", "accuracy", "precision", "recall", "f1 score"])

confusion matrix = 
 [[70294   444]
 [ 6079    61]]
accuracy =  0.9151512786492885
precision =  0.12079207920792079
recall =  0.00993485342019544
f1 score =  0.018359668924003012


In [70]:
tmp

,confusion matrix,accuracy,precision,recall,f1 score
KNN,"[[70294, 444], [6079, 61]]",0.915151,0.120792,0.009935,0.01836


In [49]:
# テストCSV読み込み

test_df = pd.read_csv("application_test.csv")
print(test_df.head())

   SK_ID_CURR NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0      100001         Cash loans           F            N               Y   
1      100005         Cash loans           M            N               Y   
2      100013         Cash loans           M            Y               Y   
3      100028         Cash loans           F            N               Y   
4      100038         Cash loans           M            Y               N   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          135000.0    568800.0      20560.5         450000.0   
1             0           99000.0    222768.0      17370.0         180000.0   
2             0          202500.0    663264.0      69777.0         630000.0   
3             2          315000.0   1575000.0      49018.5        1575000.0   
4             1          180000.0    625500.0      32067.0         625500.0   

   ... FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG

In [50]:
_factorize_categoricals(test_df, obj)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,0,0,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,0,1,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,0,1,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,0,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,0,1,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0,0,0,0,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,0,0,0,1,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,0,0,1,0,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,0,1,0,1,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [52]:
test_df = test_df.dropna(axis=1)
test_df

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,100001,0,0,0,0,0,135000.0,568800.0,450000.0,0,...,0,0,0,0,0,0,0,0,0,0
1,100005,0,1,0,0,0,99000.0,222768.0,180000.0,0,...,0,0,0,0,0,0,0,0,0,0
2,100013,0,1,1,0,0,202500.0,663264.0,630000.0,-1,...,0,0,0,0,0,0,0,0,0,0
3,100028,0,0,0,0,2,315000.0,1575000.0,1575000.0,0,...,0,0,0,0,0,0,0,0,0,0
4,100038,0,1,1,1,1,180000.0,625500.0,625500.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0,0,0,0,0,121500.0,412560.0,270000.0,0,...,0,0,0,0,0,0,0,0,0,0
48740,456222,0,0,0,1,2,157500.0,622413.0,495000.0,0,...,0,0,0,0,0,0,0,0,0,0
48741,456223,0,0,1,0,1,202500.0,315000.0,315000.0,0,...,0,0,0,0,0,0,0,0,0,0
48742,456224,0,1,0,1,0,225000.0,450000.0,450000.0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = pd.read_csv("application_train.csv")


In [57]:
test_df = pd.read_csv("application_test.csv")
print(test_df.head())

   SK_ID_CURR NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0      100001         Cash loans           F            N               Y   
1      100005         Cash loans           M            N               Y   
2      100013         Cash loans           M            Y               Y   
3      100028         Cash loans           F            N               Y   
4      100038         Cash loans           M            Y               N   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          135000.0    568800.0      20560.5         450000.0   
1             0           99000.0    222768.0      17370.0         180000.0   
2             0          202500.0    663264.0      69777.0         630000.0   
3             2          315000.0   1575000.0      49018.5        1575000.0   
4             1          180000.0    625500.0      32067.0         625500.0   

   ... FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG

In [64]:
import missingno as msno
import seaborn as sns

train_null = df.isnull().sum()[df.isnull().sum()>0].sort_values()
print(train_null.index) # 欠損値のある列を抜き出し

Index(['DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'EXT_SOURCE_2', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'NAME_TYPE_SUITE',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       'EXT_SOURCE_3', 'OCCUPATION_TYPE', 'EMERGENCYSTATE_MODE',
       'TOTALAREA_MODE', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BEGINEXPLUATATION_MEDI',
       'FLOORSMAX_AVG', 'FLOORSMAX_MEDI', 'FLOORSMAX_MODE', 'HOUSETYPE_MODE',
       'LIVINGAREA_AVG', 'LIVINGAREA_MODE', 'LIVINGAREA_MEDI',
       'ENTRANCES_MEDI', 'ENTRANCES_AVG', 'ENTRANCES_MODE', 'APARTMENTS_AVG',
       'APARTMENTS_MODE', 'APARTMENTS_MEDI', 'WALLSMATERIAL_MODE',
       'ELEVATORS_AVG', 'ELEVATORS_MEDI', 'ELEVATORS_MODE',
       'NO

In [63]:
test_null = test_df.isnull().sum()[test_df.isnull().sum()>0].sort_values()
print(test_null.index) # 欠損値のある列を抜き出し

Index(['EXT_SOURCE_2', 'AMT_ANNUITY', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'NAME_TYPE_SUITE',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       'EXT_SOURCE_3', 'OCCUPATION_TYPE', 'EXT_SOURCE_1',
       'EMERGENCYSTATE_MODE', 'TOTALAREA_MODE', 'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BEGINEXPLUATATION_AVG',
       'FLOORSMAX_AVG', 'FLOORSMAX_MEDI', 'FLOORSMAX_MODE', 'LIVINGAREA_AVG',
       'LIVINGAREA_MODE', 'LIVINGAREA_MEDI', 'ENTRANCES_AVG', 'ENTRANCES_MODE',
       'ENTRANCES_MEDI', 'HOUSETYPE_MODE', 'APARTMENTS_AVG', 'APARTMENTS_MODE',
       'APARTMENTS_MEDI', 'WALLSMATERIAL_MODE', 'ELEVATORS_MODE',
       'ELEVATORS_AVG', 'ELEVATORS_MEDI', 'NONLIVINGAREA_MEDI',
       'NONLIVINGAREA_MODE', 'NONLIVINGAREA